In [2]:
import urllib2
import tensorflow as tf
import numpy as np
import timeit
        
# let's load some data in numpy
fid = urllib2.urlopen("https://cvml.ist.ac.at/courses/DLWT_W17/data/Xtrain.txt")
Xdata = np.loadtxt(fid)
fid = urllib2.urlopen("https://cvml.ist.ac.at/courses/DLWT_W17/data/Ytrain.txt")
Ydata = np.loadtxt(fid).reshape(-1, 1)
print("data shape = ", Xdata.shape)
print("labels shape = ", Ydata.shape)

Xtrn,Ytrn = Xdata[::2],Ydata[::2] # half of the points for training
ntrn,dim = Xtrn.shape
Xval,Yval = Xdata[1::2],Ydata[1::2] # rest for model evaluation


('data shape = ', (1213, 576))
('labels shape = ', (1213, 1))


In [7]:
np.loadtxt(urllib2.urlopen("https://cvml.ist.ac.at/courses/DLWT_W17/data/Ytrain.txt")).reshape(-1,1).shape

(1213, 1)

In [2]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:800px;height:620px;border:1" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
steps_to_converge = []
nsteps = 10000
eps =10**-2

for eta in np.linspace(0.01,0.9,15):
    
    reset_graph()
    X = tf.constant(Xtrn)     
    Y = tf.constant(Ytrn)     
    X_val = tf.constant(Xval, name = "Xval")  
    Y_val = tf.constant(Yval)
    w = tf.Variable(np.random.randn(dim, 1), name="w")
    
    
    wTx = tf.matmul(X, w, name="wTx")
    YT = tf.transpose(Y, name="Ytranspose")
    XT = tf.transpose(X, name="Xtranspose")
    
    wTx_val = tf.matmul(Xval, w, name="wTx")
    YT_val = tf.transpose(Yval, name="Ytranspose")
    XT_val = tf.transpose(Xval, name="Xtranspose")
    

    loss_train = tf.reduce_mean(tf.log(1.+tf.exp(-tf.multiply(Y,wTx))), name="loss_train")
    loss_val = tf.reduce_mean(tf.log(1.+tf.exp(-tf.multiply(Y_val,wTx_val))), name="loss_val")
    
    gradient_of_loss = tf.gradients(loss_train, [w])[0] 

    update_w = w.assign( w - eta*gradient_of_loss )

    init = tf.global_variables_initializer()

    tf.get_default_graph().finalize()

    loss_train_get = 1
    loss_train_ = 0
    loss_train_temp = 1

    start_time = timeit.default_timer()
    with tf.Session() as sess:
        
        sess.run(init) # init variables
        i = 0
        while abs(loss_train_-loss_train_temp) > eps:
            i += 1
            sess.run(update_w) # execute update to w
            if i%1000 == 0:
                loss_train_temp = loss_train_get
                loss_val_temp = loss_train_get
                loss_train_get = sess.run(loss_train)
                loss_val_get = sess.run(loss_val)
                loss_train_ = loss_train_get
                loss_val_ = loss_val_get
                print("step", i, "loss(train)", loss_train_get, "loss(val)", loss_val_get, "change: ", abs(loss_train_-loss_train_temp))
    steps_to_converge.append([eta, i])
    elapsed = timeit.default_timer() - start_time 
    print "Time needed =", elapsed
print steps_to_converge

('step', 1000, 'loss(train)', 0.60676015664124505, 'loss(val)', 0.61403130299866704, 'change: ', 0.39323984335875495)
('step', 2000, 'loss(train)', 0.46471474031179, 'loss(val)', 0.48885199510625277, 'change: ', 0.14204541632945505)
('step', 3000, 'loss(train)', 0.40084972036726768, 'loss(val)', 0.43851711639902452, 'change: ', 0.063865019944522317)
('step', 4000, 'loss(train)', 0.36214658774918607, 'loss(val)', 0.41030030520519473, 'change: ', 0.038703132618081615)
('step', 5000, 'loss(train)', 0.33543158743835771, 'loss(val)', 0.3917488184524896, 'change: ', 0.026715000310828352)
('step', 6000, 'loss(train)', 0.31561430863936141, 'loss(val)', 0.37838800112260351, 'change: ', 0.019817278798996307)
('step', 7000, 'loss(train)', 0.30016586260430794, 'loss(val)', 0.36818126544516866, 'change: ', 0.015448446035053465)
('step', 8000, 'loss(train)', 0.28766365601574811, 'loss(val)', 0.3600607171687904, 'change: ', 0.012502206588559828)
('step', 9000, 'loss(train)', 0.27724880301322724, 'los

('step', 3000, 'loss(train)', 0.2415511087753838, 'loss(val)', 0.65618437256174356, 'change: ', 0.073978842453296806)
('step', 4000, 'loss(train)', 0.11666300312260573, 'loss(val)', 0.50868821167806177, 'change: ', 0.12488810565277807)
('step', 5000, 'loss(train)', 2.9155127499208415, 'loss(val)', 3.4805079633660645, 'change: ', 2.7988497467982358)
('step', 6000, 'loss(train)', 0.082633051133887442, 'loss(val)', 0.47461273724942166, 'change: ', 2.8328796987869542)
('step', 7000, 'loss(train)', 0.073814136957731133, 'loss(val)', 0.45907693600493288, 'change: ', 0.0088189141761563089)
Time needed = 2.95722293854
('step', 1000, 'loss(train)', 0.2745680289570922, 'loss(val)', 0.60866433494834682, 'change: ', 0.7254319710429078)
('step', 2000, 'loss(train)', 3.078209988564768, 'loss(val)', 3.6185801305194896, 'change: ', 2.8036419596076758)
('step', 3000, 'loss(train)', 0.14415178627813455, 'loss(val)', 0.546211472129709, 'change: ', 2.9340582022866335)
('step', 4000, 'loss(train)', 0.20090

In [4]:
steps_to_converge = []
for eta in np.linspace(0.01,0.9,15):
    reset_graph()
    X = tf.placeholder(dtype=tf.float32, name="X")
    Y = tf.placeholder(dtype=tf.float32, name="Y")
    w = tf.Variable(tf.zeros(shape=(dim,1), dtype=tf.float32), name="w")
    nsteps = 10000
    eps =10**-2

    wTx = tf.matmul(X, w, name="wTx")

    YT = tf.transpose(Y, name="Ytranspose")
    XT = tf.transpose(X, name="Xtranspose")

    loss = tf.reduce_mean(tf.log(1.+tf.exp(-tf.multiply(Y,wTx, name = "YwTx"))), name="loss")

    gradient_of_loss = tf.gradients(loss, [w])[0]  # <-- automatic differentiation

    update_w = w.assign( w - eta*gradient_of_loss )

    init = tf.global_variables_initializer()

    tf.get_default_graph().finalize()

    loss_train = 1
    loss_train_ = 0
    loss_train_temp = 1

    start_time = timeit.default_timer()
    with tf.Session() as sess:
        
        sess.run(init) # init variables
        i = 0
        while abs(loss_train_-loss_train_temp) > eps:
            i += 1
            sess.run(update_w, feed_dict={X: Xtrn, Y:Ytrn}) # execute update to w
            if i%1000 == 0:
                loss_train_temp = loss_train
                loss_val_temp = loss_train
                loss_train = sess.run(loss, feed_dict={X: Xtrn, Y:Ytrn})
                loss_val = sess.run(loss, feed_dict={X: Xval, Y:Yval})
                loss_train_ = loss_train
                loss_val_ = loss_val
                print("step", i, "loss(train)", loss_train, "loss(val)", loss_val, "change: ", abs(loss_train_-loss_train_temp))
    steps_to_converge.append([eta, i])
    elapsed = timeit.default_timer() - start_time 
    print "Time needed =", elapsed
print steps_to_converge

('step', 1000, 'loss(train)', 0.46168852, 'loss(val)', 0.46896544, 'change: ', 0.53831148147583008)
('step', 2000, 'loss(train)', 0.39271244, 'loss(val)', 0.40877038, 'change: ', 0.068976074)
('step', 3000, 'loss(train)', 0.35334131, 'loss(val)', 0.3767657, 'change: ', 0.039371133)
('step', 4000, 'loss(train)', 0.32690698, 'loss(val)', 0.35637325, 'change: ', 0.026434332)
('step', 5000, 'loss(train)', 0.30752471, 'loss(val)', 0.34200528, 'change: ', 0.019382268)
('step', 6000, 'loss(train)', 0.29248235, 'loss(val)', 0.3312158, 'change: ', 0.015042365)
('step', 7000, 'loss(train)', 0.280334, 'loss(val)', 0.32275388, 'change: ', 0.01214835)
('step', 8000, 'loss(train)', 0.27023056, 'loss(val)', 0.31590706, 'change: ', 0.010103434)
('step', 9000, 'loss(train)', 0.26163599, 'loss(val)', 0.31023595, 'change: ', 0.0085945725)
Time needed = 6.398209095
('step', 1000, 'loss(train)', 0.27499416, 'loss(val)', 0.31914929, 'change: ', 0.72500583529472351)
('step', 2000, 'loss(train)', 0.22827393, 

('step', 3000, 'loss(train)', 0.24125075, 'loss(val)', 0.67880714, 'change: ', 0.023025066)
('step', 4000, 'loss(train)', 0.15214433, 'loss(val)', 0.61680573, 'change: ', 0.089106426)
('step', 5000, 'loss(train)', 0.10884734, 'loss(val)', 0.61131561, 'change: ', 0.043296985)
('step', 6000, 'loss(train)', 0.096123442, 'loss(val)', 0.62423825, 'change: ', 0.0127239)
('step', 7000, 'loss(train)', 0.09384194, 'loss(val)', 0.65196157, 'change: ', 0.0022815019)
Time needed = 5.3418879509
[[0.01, 9000], [0.073571428571428565, 6000], [0.13714285714285715, 5000], [0.20071428571428573, 5000], [0.26428571428571429, 5000], [0.32785714285714285, 5000], [0.39142857142857146, 6000], [0.45500000000000002, 5000], [0.51857142857142857, 6000], [0.58214285714285718, 5000], [0.64571428571428569, 7000], [0.7092857142857143, 6000], [0.77285714285714291, 6000], [0.83642857142857141, 6000], [0.90000000000000002, 7000]]


In [5]:
show_graph(tf.get_default_graph())

In [6]:
# Analytic solution
reset_graph()
nsteps = 10000
eta = 0.26
X = tf.placeholder(dtype=tf.float32, name="X")
Y = tf.placeholder(dtype=tf.float32, name="Y")
w = tf.Variable(tf.zeros(shape=(dim,1), dtype=tf.float32), name="w")

wTx = tf.matmul(X, w, name="wTx")

YT = tf.transpose(Y, name="Ytranspose")
XT = tf.transpose(X, name="Xtranspose")
p = 1./(1.+tf.exp(-tf.multiply(Y,wTx, name = "YTwTx")))

loss = tf.reduce_mean(tf.log((1.+tf.exp(-tf.multiply(Y,wTx)))), name="loss")

gradient_of_loss = - tf.matmul(XT, tf.multiply(tf.div(Y,(1 + tf.exp(-tf.multiply(Y,wTx)))),tf.exp(-tf.multiply(Y,wTx)))) / ntrn 

update_w = w.assign( w - eta*gradient_of_loss )

init = tf.global_variables_initializer()

tf.get_default_graph().finalize()

start_time = timeit.default_timer()
with tf.Session() as sess:
    sess.run(init) # init variables
    for i in range(nsteps):
        sess.run(update_w, feed_dict={X: Xtrn, Y:Ytrn}) # execute update to w
        if i%1000 == 0:
            loss_train = sess.run(loss, feed_dict={X: Xtrn, Y:Ytrn})
            loss_val = sess.run(loss, feed_dict={X: Xval, Y:Yval})
            print("step", i, "loss(train)", loss_train, "loss(val)", loss_val)
elapsed = timeit.default_timer() - start_time 
print "Time needed =", elapsed

('step', 0, 'loss(train)', 0.71049929, 'loss(val)', 0.69179559)
('step', 1000, 'loss(train)', 0.19493896, 'loss(val)', 0.31109157)
('step', 2000, 'loss(train)', 0.15199344, 'loss(val)', 0.27827641)
('step', 3000, 'loss(train)', 0.13512972, 'loss(val)', 0.27664959)
('step', 4000, 'loss(train)', 0.12417734, 'loss(val)', 0.27362087)
('step', 5000, 'loss(train)', 0.11656693, 'loss(val)', 0.27370769)
('step', 6000, 'loss(train)', 0.11050525, 'loss(val)', 0.27491102)
('step', 7000, 'loss(train)', 0.10540012, 'loss(val)', 0.27674046)
('step', 8000, 'loss(train)', 0.1009809, 'loss(val)', 0.27896139)
('step', 9000, 'loss(train)', 0.097087055, 'loss(val)', 0.28142971)
Time needed = 23.1403450966
